In [1]:
#---------------------------------------------------------------------------------------------------------------------------
# ABS Population Data - raw data in excel multi-sheet file, sheet G04
# Process ABS 'General Community Profile' (GCP) data downloaded from ABS online site as excel files into 'datapandas/' directory.
# Each State and Territories of Australia has their own GCP excel multi-sheet file.
# Each file has sheet 'G04' which contains the count of persons (population) data by Age by Sex at the time of 2021 census.
# Each file has been renamed as follows:
#    'GCP1GeneralCommunityProfileNSW.xlsx', 'GCP2GeneralCommunityProfileVIC.xlsx', 'GCP3GeneralCommunicationProfileQLD.xlsx'
#    'GCP4GeneralCommunityProfileSA.xlsx', 'GCP5GeneralCommunityProfileWA.xlsx', 'GCP6GeneralCommunityProfileTAS.xlsx'
#    GCP7GeneralCommunityProfileNT.xlsx', 'GCP8GeneralCommunityProfileACT.xlsx', 'GCP9GeneralCommunityProfileOTH.xlsx'
# All GCP files will be read and its G04 sheet processed by pandas to create two reformatted population count by Age by Sex
# multi-sheet excel (by State) files.
# Output files:
#  1. 'ABS2021AgeBySex.xlsx' - file containing the lowest level population count by "Age by Sex" for ages 0,1,2...up to 79.
#  2. 'ABS2021AgeBySexGrp.xlsx' - file containing grouped level population count by "Age by Sex".
#          ABS does not supply lowest level data for ages 80 years onwards, only grouped count is supplied:
#          '80-84 years', '85-89 years', '90-94 years', '95-99 years', '100 years and over'
# NOTE: Row 9 of each of the GCP's G04 sheet excel state FILES WERE EDITED to put labels/headers so that data can be
#       properly loaded by pandas. IT IS JUST A ONE-LINER MINOR EDIT PER FILE.
#---------------------------------------------------------------------------------------------------------------------------
import os               # needed for directory and file listing
import re               # import regular expressions module for string search
import pandas as pd     # import pandas

# Define variables
datapath = 'datapandas/'  # downloaded ABS data location under pycharm root directory
substring = "GCP"  # for use in processing only ABS GCP files
sheetFlag=0  # this flag is for the saving of multiple sheets in the output excel file saved from manipulated dataftrames.
outfilename1="ABS2021AgeBySex.xlsx"     #output file
outfilename2="ABS2021AgeBySexGrp.xlsx"  #output file

# Read datasource directory and loop through and process each ABS GCP State excel file.
with os.scandir(datapath) as entries:
    for entry in entries:
        if entry.is_file() and re.search(substring, entry.name):  # loop through all the State GCP files
            #print(entry.name)

# Create a dataframe from a GCP sheet G04 excel file.
# Skip top 8 rows when loading as they are only heading information
            df=pd.read_excel(datapath+entry.name,sheet_name='G04',skiprows=8)
            #print(df)

# Drop/delete unnecessary columns in the dataframe.
# To specify we want to drop column, we need to provide axis=1 as an argument to drop function.
# Parameter inplace=True tells pandas to update the same dataframe therefore losing what is being dropped
# Persons1 and Persons2 are Totals which are not needed. Blank1 and Blank2 are columns with spaces as values.
            df.drop(['Blank1', 'Blank2','Persons1','Persons2','Persons3'], axis=1,inplace=True)
            #print(df)

# Analysing the data at this point, "Age1" column has sub-totals and string values that need to be deleted.
# Use to_numeric function to convert non-numeric data into NaN in "Age1" column - specifying 'coerce' in the errors
# argument will replace non-numeric values with NaN.
            df['Age1'] = pd.to_numeric(df['Age1'], errors='coerce')
            #print('Here1', df, '\n')

# Drop/delete all rows with values of NaN in column 'Age1'
            df.dropna(subset=['Age1'],inplace=True)
            #print('Here1', df, '\n')

# Reset index without adding the old 'index' column by specifying drop=True
            df.reset_index(drop=True, inplace=True)
            #print('Here1', '\n', df, '\n')

# The data at this point needs to be split by columns into three chunks.
# First columns split.
            coldata1 = df.loc[:,'Age1':'Females1']
# Rename columns to proper names, ie, "Age1" to "Age" to be able to concatenate by column later.
            coldata1.rename({'Age1': 'Age','Males1': 'Males', 'Females1': 'Females'}, axis = "columns", inplace = True)
            #print(coldata1)

# Second coulmns split.
            coldata2 = df.loc[:,'Age2':'Females2']
# Rename columns to proper names, ie, "Age2" to "Age" to be able to concatenate by column later.
            coldata2.rename({'Age2': 'Age','Males2': 'Males', 'Females2': 'Females'}, axis = "columns", inplace = True)
            #print(coldata2)

# Third coulmns split.
            coldata3 = df.loc[:,'Age3':'Females3']
# Rename columns to proper names, ie, "Age3" to "Age" to be able to concatenate by column later.
            coldata3.rename({'Age3': 'Age','Males3': 'Males', 'Females3': 'Females'}, axis = "columns", inplace = True)
            #print(coldata3)

# Now, concatenate the three split dataframes into one single dataframe appending data into rows based on column labels.
# To reset the index after the concat, set the parameter ignore_index=True.
# axis=0 means concat will work on rows; axis=1 means concat will work on columns
            df1 = pd.concat([coldata1, coldata2, coldata3], ignore_index=True, sort=False, axis=0)
            #print('Here1', '\n', df1.iloc[27:35,:], '\n')

# Extract lowest level data by "Age by Sex" for this State
            dfStateAgeBySex=df1.iloc[0:80]   # Ages 0 to 79
            #print(dfStateAgeBySex)

# Extract grouped data "Age by Sex" for this State (ABS does not supply lowest level data for these age groups).
            dfStateAgeBySexGrp=df1.iloc[80:85] # grouped Ages as per ABS
            #print(dfStateAgeBySexGrp)

# Create excel output files
            stateList = ['NSW','VIC','QLD','SA','WA','TAS','NT','ACT','OTH']  # State is sheet name
            for state in stateList:  # Check which State were processing
                if re.search(state, entry.name):
                    if sheetFlag==0:  # create the first sheet in the output excel file
# Save dataframe to excel file with state as sheet name (first sheet).
                        #print("here1=", state)
                        # first sheet lowest level data, do not save index
                        dfStateAgeBySex.to_excel(datapath+outfilename1, sheet_name=state,index=False)
                        # first sheet grouped data per ABS, do not save index
                        dfStateAgeBySexGrp.to_excel(datapath+outfilename2, sheet_name=state,index=False)
                        sheetFlag=1
                    else:  # append sheet to the excel file
# Save dataframe to excel file with state as sheet name (append sheet).
                        #print("here2=", state)
                        # append sheet lowest level data, do not save index
                        with pd.ExcelWriter(datapath+outfilename1, mode='a') as writer:
                            dfStateAgeBySex.to_excel(writer, sheet_name=state,index=False)
                        # append sheet grouped data as per ABS, do not save index
                        with pd.ExcelWriter(datapath+outfilename2, mode='a') as writer:
                            dfStateAgeBySexGrp.to_excel(writer, sheet_name=state,index=False)
                    break
            #print('Finished statelist for a GCP file')
    #print('Finished GCP files')

#end of file